In [1]:
import matplotlib.pyplot as plt
import numpy as np

"""In order to provide a better presentation of the graphs we use the rcParams options shown below."""

import matplotlib
matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams['figure.figsize'] = (11.0, 8.0)

In [2]:
import matplotlib.pyplot as plt
from numpy import linalg as LA
import scipy.special

import time
from scipy.integrate import odeint
from scipy.special import zeta
from random import choices

from numpy import sin, cos, power


In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import numpy as np

from scipy.optimize import minimize
from qutip import *
from qutip.piqs import *

import matplotlib.animation as animation
from IPython.display import HTML
from IPython.core.display import Image, display

In [4]:
"""Our system of differential equations belongs to the complex space, that is why we implemented the code suggested in"""
"""https://stackoverflow.com/questions/19910189/scipy-odeint-with-complex-initial-values."""

import time
import numpy as np
from scipy.integrate import odeint
from scipy.special import zeta
from random import choices

def odeintz(func, z0, t, **kwargs):
   
    
    """An odeint-like function for complex valued differential equations.

    Inputs:
    ----------
      -func: function associated to dr/dt=f(x;t), where x is the set of parameters and variables to be determined
      -z0: 1d array with length N*(5N-1)/2
      -t: 1d array from t=0 to t=tf (parameter set by the user)
      - **kwargs: keyword arguments related with external functions to be used in odeint
    
    Return:
      -z: multivariable array with the solution of the differential equation associated with each variable"""

    # Disallow Jacobian-related arguments.
    _unsupported_odeint_args = ['Dfun', 'col_deriv', 'ml', 'mu']
    bad_args = [arg for arg in kwargs if arg in _unsupported_odeint_args]
    if len(bad_args) > 0:
        raise ValueError("The odeint argument %r is not supported by "
                         "odeintz." % (bad_args[0],))

    # Make sure z0 is a numpy array of type np.complex128.
    z0 = np.array(z0, dtype=np.complex128, ndmin=1)

    def realfunc(x, t, *args):
        z = x.view(np.complex128)
        dzdt = func(z, t, *args)
        # func might return a python list, so convert its return
        # value to an array with type np.complex128, and then return
        # a np.float64 view of that array.
        return np.asarray(dzdt, dtype=np.complex128).view(np.float64)

    result = odeint(realfunc, z0.view(np.float64), t, **kwargs)

    if kwargs.get('full_output', False):
        z = result[0].view(np.complex128)
        infodict = result[1]
        return z, infodict
    else:
        z = result.view(np.complex128)
        return z

## Euler-Maruyama

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import random, zeros

from numba import jit

dt = 0.002 # Time step.
T = 20 # Total time.
n = int(T / dt) # Number of time steps.
steps=int(T/dt)

times = np.linspace(0., T, n)


J=1
N=10
    
refg     = 1
refomega = 2
refdelta=  0.5
refkappa= 0.3
refgamma= 0.4

    
Omega =refomega*J
g_coef=refg*J
Delta=refdelta*J
kappa=refkappa*J
Gamma_phi=refgamma*J
    

Gamma_du=0  #Fixed


Stochastic=1   #Is the process stochastic?
sqrt_N=np.sqrt(N)
sqrt_2phi=np.sqrt(2*Gamma_phi)

theta=0   #Initial state
phi=0



#set the initial values due to the measurements in the state |\psi>=cos(\theta/2)|0>+sin(\theta/2)|1>

from scipy import stats
mk = np.array([-1,1])


xp= (0.5*(1-np.sin(theta)*np.cos(phi)),0.5*(1+np.sin(theta)*np.cos(phi)))
yp= (0.5*(1-np.sin(theta)*np.sin(phi)),0.5*(1+np.sin(theta)*np.sin(phi)))
zp = ((np.cos(theta/2))**2, (np.sin(theta/2))**2)


#xp=(0.5,0.5)
#yp=(0.5,0.5)
#zp=(1,0)



custmx = stats.rv_discrete(name='custmx', values=(mk, xp))
custmy = stats.rv_discrete(name='custmy', values=(mk, yp))
custmz = stats.rv_discrete(name='custmz', values=(mk, zp))




@jit

def trayectory(N=N):
    sqrtdt = np.sqrt(dt)*Stochastic
    
    Results =zeros((3*N+2, n))
    Results[3*N][0]  =np.random.normal(0, 0.5)
    Results[3*N+1][0]=np.random.normal(0, 0.5)
    
    for j in range(N):
        
        a=custmx.rvs()
        b=custmy.rvs()
        c=custmz.rvs()
        
        Results[j][0]    =a  
        Results[j+N][0]  =b
        Results[j+2*N][0]=c
        
    
    for t in range(n-1):
        
        Normal_VA_1=random.randn() #np.random.normal(0, 1)
        Normal_VA_2=random.randn() #np.random.normal(0, 1)
        
        Results[3*N][t+1]   = Results[3*N][t]  +dt*Delta*Results[3*N+1][t]+\
            -kappa*Results[3*N][t]*dt+np.sqrt(kappa/2)* sqrtdt * Normal_VA_1
        Results[3*N+1][t+1] = Results[3*N+1][t]-dt*Delta* Results[3*N][t]+\
            -kappa*Results[3*N+1][t]*dt+np.sqrt(kappa/2)*sqrtdt * Normal_VA_2
        
        Normal_VA=random.randn()   #np.random.normal(0, 1)
        
        
        for i in range(N): 
            
            Results[i][t+1]    = Results[i][t]+dt*(-2*g_coef/sqrt_N)*Results[3*N+1][t]*Results[i+2*N][t]-\
                dt*Gamma_phi*Results[i][t]-sqrt_2phi*Results[i+N][t]* sqrtdt * Normal_VA
            
            Results[i+N][t+1]  = Results[i+N][t]+dt*(-2*g_coef/sqrt_N)*Results[3*N][t]*Results[i+2*N][t]-dt*Omega*Results[i+2*N][t]+\
                -dt*Gamma_phi*Results[i+N][t]+sqrt_2phi*Results[i][t]* sqrtdt * Normal_VA
            
            Results[i+2*N][t+1]= Results[i+2*N][t]+dt*(2*g_coef/sqrt_N)*Results[3*N][t]*Results[i+N][t]+\
                dt*(2*g_coef/sqrt_N)*Results[3*N+1][t]*Results[i][t]+dt*Omega*Results[i+N][t]
            
            Results[3*N][t+1]   += -(g_coef/(2*sqrt_N))*Results[i+N][t]*dt
            Results[3*N+1][t+1] += -(g_coef/(2*sqrt_N))*Results[i][t]*dt
            
    return Results


In [6]:
from numpy import mean
from numpy import linalg

from numba import jit
@jit

def summary_stat(Res,N=N,n=n):
    Results =zeros((14, n))
    count=0
    while count<n:
        a,b,c=Res[0:N, count:count+1],Res[N:2*N, count:count+1],Res[2*N:3*N, count:count+1]
        
        Results[0][count],Results[1][count],Results[2][count]=N*mean(a)/2,N*mean(b)/2,N*mean(c)/2
        Results[3][count]=0.25*(  (N*mean(a))**2- (linalg.norm(a,2))**2 )
        Results[4][count]=0.25*(  (N*mean(b))**2- (linalg.norm(b,2))**2 )
        Results[5][count]=0.25*(  (N*mean(c))**2- (linalg.norm(c,2))**2 )
        Results[6][count]=(N*mean(a)/2)**2
        Results[7][count]=(N*mean(b)/2)**2
        Results[8][count]=(N*mean(c)/2)**2
        
        alpha_x=mean(Res[3*N:3*N+1, count:count+1])
        alpha_y=mean(Res[3*N+1:3*N+2, count:count+1])
        
        alpha_cuad=alpha_x**2+alpha_y**2
        
        Results[9][count]=alpha_cuad  #np.power(alpha_x,2)+np.power(alpha_y,2)-0.5
        Results[10][count]=alpha_cuad**2-2*alpha_cuad+0.5

        
        
        
        Results[11][count]=Results[0][count]*Results[1][count]    #xy
        Results[12][count]=Results[1][count]*Results[2][count]    #yz
        Results[13][count]=Results[2][count]*Results[0][count]    #zx
        
        count=count+1
        
    
    return Results   

In [7]:
from numpy import add
from numba import jit



itera=5000
@jit

def averages(itera=itera,N=N,n=n,Stochastic=Stochastic):
    Results =zeros((14, n))
    for j in range(itera):
        Trayec=trayectory(N=N)
        Results=add(Results,summary_stat(Trayec,N,n))
    
    return Results/itera
    
    
    
    

In [ ]:
t0=time.time()

itera=2500

Result_varDDTWA_EM=averages(itera=itera,N=N,n=n,Stochastic=1)
#Result_varDTWA=averages(itera=itera,N=N,n=n,Stochastic=0)

t1=time.time()

print(t1-t0)



<ipython-input-7-a9b5efe0048a>:7: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "averages" failed type inference due to: Invalid use of type(CPUDispatcher(<function trayectory at 0x0000029737BD5790>)) with parameters (N=int64)

During: resolving callee type: type(CPUDispatcher(<function trayectory at 0x0000029737BD5790>))
During: typing of call at <ipython-input-7-a9b5efe0048a> (12)


File "<ipython-input-7-a9b5efe0048a>", line 12:
def averages(itera=itera,N=N,n=n,Stochastic=Stochastic):
    <source elided>
    for j in range(itera):
        Trayec=trayectory(N=N)
        ^

  @jit
<ipython-input-7-a9b5efe0048a>:7: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "averages" failed type inference due to: cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-7-a9b5efe0048a>", line 11:
def averages(itera=itera,N=N,n=n,Stochastic=Stochastic):

## Stratonovich-Heun

In [ ]:
#### IN PREPARATION


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import random, zeros

from numba import jit

n = int(T / dt) # Number of time steps.

times = np.linspace(0., T, n)

sq_kdiv2=np.sqrt(kappa/2)


@jit

def trayectorystra(N=N,Stochastic=Stochastic):
    sqrtdt = np.sqrt(dt)*Stochastic
        
    #Results[j][0]    =a  
    #Results[j+N][0]  =b
    #Results[j+2*N][0]=c
    
    Results = zeros((3*N+2, n))
    Rbar    = zeros((3*N+2, n))
    
    Results[3*N][0]  =np.random.normal(0, 0.5)
    Results[3*N+1][0]=np.random.normal(0, 0.5)
    

    for j in range(N):
        
        a=custmx.rvs()
        b=custmy.rvs()
        c=custmz.rvs()
        Results[j][0]    =a
        Results[j+N][0]  =b
        Results[j+2*N][0]=c


    for t in range(n-1):
        
        Normal_VA =random.randn()
        Normal_VA1=random.randn()
        Normal_VA2=random.randn()
        
        Rbar[3*N][t]   = Results[3*N][t]  +dt*(Delta*Results[3*N+1][t]-kappa*Results[3*N][t]) +sq_kdiv2*sqrtdt*Normal_VA1
        Rbar[3*N+1][t] = Results[3*N+1][t]+dt*(-Delta*Results[3*N][t]-kappa*Results[3*N+1][t])+sq_kdiv2*sqrtdt*Normal_VA2
        
        for i1 in range(N):

            Rbar[i1][t]     = Results[i1][t]    +dt*(-2*g_coef/sqrt_N)*Results[3*N+1][t]*Results[i1+2*N][t]-\
                (sqrt_2phi) *Results[i1+N][t]* sqrtdt * Normal_VA
            Rbar[i1+N][t]   = Results[i1+N][t]  +dt*(-2*g_coef/sqrt_N)*Results[3*N][t]*Results[i1+2*N][t]-\
                dt*Omega*Results[i1+2*N][t] + sqrt_2phi*Results[i1][t]* sqrtdt * Normal_VA
            Rbar[i1+2*N][t] = Results[i1+2*N][t]+ dt*Omega*Results[i1+N][t] +\
                dt*(2*g_coef/sqrt_N)*(Results[3*N][t]*Results[i1+N][t]+Results[3*N+1][t]*Results[i1][t])
            
            Rbar[3*N][t]   +=dt*(g_coef/(2*sqrt_N))*Results[i1+N][t]
            Rbar[3*N+1][t] +=dt*(-g_coef/(2*sqrt_N))*Results[i1][t]
            
#-------------------------------------------------------------------------------------------------------            
        
        Results[3*N][t+1]   = Results[3*N][t]-dt*kappa*(Results[3*N][t]+Rbar[3*N][t])*0.5+\
            dt*0.5*Delta*(Results[3*N+1][t] + Rbar[3*N+1][t])+sq_kdiv2*sqrtdt*Normal_VA1
        Results[3*N+1][t+1] = Results[3*N+1][t]-dt*kappa*(Results[3*N+1][t]+Rbar[3*N+1][t])*0.5-\
            dt*0.5*Delta*(Results[3*N][t] + Rbar[3*N][t])+sq_kdiv2*sqrtdt*Normal_VA2
        
        for i in range(N): 
                        
            fxn   = (-2*g_coef/sqrt_N)*Results[3*N+1][t]*Results[i+2*N][t]
            Gxn   = -(sqrt_2phi)*Results[i+N][t]
            fxbarn= (-2*g_coef/sqrt_N)*Rbar[3*N+1][t]*Rbar[i+2*N][t]
            Gxbarn= -(sqrt_2phi)*Rbar[i+N][t] 
            
            
            fyn   = (-2*g_coef/sqrt_N)*Results[3*N][t]*Results[i+2*N][t]-Omega*Results[i+2*N][t]
            Gyn   = (sqrt_2phi)*Results[i][t]
            fybarn= (-2*g_coef/sqrt_N)*Rbar[3*N][t]*Rbar[i+2*N][t]-Omega*Rbar[i+2*N][t]
            Gybarn= (sqrt_2phi)*Rbar[i][t]
            
            
            fzn   = (2*g_coef/sqrt_N)*(Results[3*N][t]*Results[i+N][t]+Results[3*N+1][t]*Results[i][t])+Omega*Results[i+N][t]
            Gzn   = 0
            fzbarn= (2*g_coef/sqrt_N)*(Rbar[3*N][t]*Rbar[i+N][t]+Rbar[3*N+1][t]*Rbar[i][t])+Omega*Rbar[i+N][t]
            Gzbarn= 0

#-------------------------------------------------------------------------------------------------------            
        
            Results[i][t+1]     = Results[i][t]    +dt*0.5*(fxn+fxbarn)+0.5*(Gxn + Gxbarn)* sqrtdt * Normal_VA
            Results[i+N][t+1]   = Results[i+N][t]  +dt*0.5*(fyn+fybarn)+0.5*(Gyn + Gybarn)* sqrtdt * Normal_VA
            Results[i+2*N][t+1] = Results[i+2*N][t]+dt*0.5*(fzn+fzbarn)+0.5*(Gzn + Gzbarn)* sqrtdt * Normal_VA
            
            Results[3*N][t+1]   +=  dt*(g_coef/(2*sqrt_N))*0.5*(Results[i+N][t]+Rbar[i+N][t])
            Results[3*N+1][t+1] += -dt*(g_coef/(2*sqrt_N))*0.5*(Results[i][t]+Rbar[i][t])
            
    return Results, Rbar

In [ ]:
from numpy import mean
from numpy import linalg

from numba import jit
@jit

def summary_stat(Res,N=N,n=n):
    Results =zeros((6, n))
    count=0
    while count<n:
        a,b,c=Res[0:N, count:count+1],Res[N:2*N, count:count+1],Res[2*N:3*N, count:count+1]
        
        Results[0][count],Results[1][count],Results[2][count]=N*mean(a)/2,N*mean(b)/2,N*mean(c)/2
        Results[3][count]=0.25*(  (N*mean(a))**2- (linalg.norm(a,2))**2 )
        Results[4][count]=0.25*(  (N*mean(b))**2- (linalg.norm(b,2))**2 )
        Results[5][count]=0.25*(  (N*mean(c))**2- (linalg.norm(c,2))**2 )
        
        count=count+1
        
    
    return Results
    
    

In [ ]:
from numpy import add
from numba import jit



itera=1000
@jit

def averages(itera=itera,N=N,n=n,Stochastic=Stochastic):
    Results =zeros((6, n))
    for j in range(itera):
        Trayec=trayectorystra(N,Stochastic)[0]
        Results=add(Results,summary_stat(Trayec,N,n))
    
    return Results/itera
    
    
    
    

In [ ]:
t0=time.time()

itera=100

Result_varDDTWA_SH=averages(itera=itera,N=N,n=n,Stochastic=1)
#Result_varDTWA=averages(itera=itera,N=N,n=n,Stochastic=0)

t1=time.time()

print(t1-t0)



## Results DDTWA

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Quantum Dynamics Photon Number', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle a^{\dagger}a \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[9]-0.5,"--",label="$a^{\dagger}a$ DDTWA EM")

#plt.plot(times, Result_varDDTWA_SH[9]-0.5,"--",label="$a^{\dagger}a$ DDTWA SH")


#plt.legend(fontsize = label_size)
plt.legend()
plt.show()
plt.close()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
plt.title(r'Quantum Evolution of $S_k$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle S_k \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[0],"--",label="$S_x$ DDTWA EM")

plt.plot(times, Result_varDDTWA_EM[1],"--",label="$S_y$ DDTWA EM")

plt.plot(times, Result_varDDTWA_EM[2],"--",label="$S_z$ DDTWA EM")

#plt.plot(times, Result_varDDTWA_SH[0],"--",label="$S_x$ DDTWA SH")

#plt.plot(times, Result_varDDTWA_SH[1],"--",label="$S_y$ DDTWA SH")

#plt.plot(times, Result_varDDTWA_SH[2],"--",label="$S_z$ DDTWA SH")




#plt.legend(fontsize = label_size)
plt.legend(loc="upper right")
plt.show()
plt.close()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Quantum Dynamics $a^{\dagger}a^{\dagger}aa$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle a^{\dagger}a^{\dagger} aa \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[10],"--",label="$a^{\dagger}a^{\dagger}aa$ DDTWA EM")
#plt.plot(times, Result_varDDTWA_SH[10],"--",label="$a^{\dagger}a^{\dagger}aa$ DDTWA SH")




#plt.legend(fontsize = label_size)
plt.legend()
plt.show()
plt.close()

In [ ]:
# DESCRIPTION OF MOMENTS OF S_k

Sxx=N/4+  Result_varDDTWA_EM[3+0]
Syy=N/4+  Result_varDDTWA_EM[3+1]
Szz=N/4+  Result_varDDTWA_EM[3+2]




In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Quantum Dynamics $\Delta (S_k^2)$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\Delta (S_k^2) (t)$', fontsize = label_size)


axEM=N/4+  Result_varDDTWA_EM[3+0]-(Result_varDDTWA_EM[0])**2
ayEM=N/4+  Result_varDDTWA_EM[3+1]-(Result_varDDTWA_EM[1])**2
azEM=N/4+  Result_varDDTWA_EM[3+2]-(Result_varDDTWA_EM[2])**2


#Use for unitary dynamics solely

#ax1EM=np.array(Result_varDDTWA_EM[6+0])-np.array((Result_varDDTWA_EM[0])**2)
#ay1EM=np.array(Result_varDDTWA_EM[6+1])-np.array((Result_varDDTWA_EM[1])**2)
#az1EM=np.array(Result_varDDTWA_EM[6+2])-np.array((Result_varDDTWA_EM[2])**2)




#axSH=N/4+  Result_varDDTWA_SH[3+0]-(Result_varDDTWA_SH[0])**2
#aySH=N/4+  Result_varDDTWA_SH[3+1]-(Result_varDDTWA_SH[1])**2
#azSH=N/4+  Result_varDDTWA_SH[3+2]-(Result_varDDTWA_SH[2])**2


#Use for unitary dynamics solely

#ax1SH=np.array(Result_varDDTWA_SH[6+0])-np.array((Result_varDDTWA_SH[0])**2)
#ay1SH=np.array(Result_varDDTWA_SH[6+1])-np.array((Result_varDDTWA_SH[1])**2)
#az1SH=np.array(Result_varDDTWA_SH[6+2])-np.array((Result_varDDTWA_SH[2])**2)



plt.plot(times,axEM,"-.",label="DDTWA $(\Delta S_x)^2 $ EM")
plt.plot(times,ayEM,"-.",label="DDTWA $(\Delta S_y)^2 $ EM")
plt.plot(times,azEM,"-.",label="DDTWA $(\Delta S_z)^2 $ EM")

#plt.plot(times,axSH,"-.",label="DDTWA $(\Delta S_x)^2 $ SH")
#plt.plot(times,aySH,"-.",label="DDTWA $(\Delta S_y)^2 $ SH")
#plt.plot(times,azSH,"-.",label="DDTWA $(\Delta S_z)^2 $ SH")


plt.legend()

plt.legend(loc="upper right")

In [ ]:
f = lambda x, Sxx, Syy, Szz, Sxys, Syzs, Szxs, Sx, Sy, Sz: Sxx*power(cos(x[0])*sin(x[1]),2)+\
    Syy*power(sin(x[0])*sin(x[1]),2)+Szz*power(cos(x[1]),2)+Sxys*sin(2*x[0])*power(sin(x[1]),2)+\
    Syzs*sin(2*x[1])*sin(x[0])+Szxs*sin(2*x[1])*cos(x[0])


Squeez=[]

Arraytime=[]

separ=10


for j in range(int(steps/separ)):
    
    j=separ*j
    
    Sx= Result_varDDTWA_EM[0][j]
    Sy= Result_varDDTWA_EM[1][j]
    Sz= Result_varDDTWA_EM[2][j]
    
    Sxx= N/4+Result_varDDTWA_EM[3][j]
    Syy= N/4+Result_varDDTWA_EM[4][j]
    Szz= N/4+Result_varDDTWA_EM[5][j]
    
    Sxy= Result_varDDTWA_EM[11][j]
    Syz= Result_varDDTWA_EM[12][j]
    Szx= Result_varDDTWA_EM[13][j]
    
    cons = ({'type': 'eq', 'fun': lambda x: Sx*cos(x[0])*sin(x[1])+Sy*sin(x[0])*sin(x[1])+Sz*cos(x[1])})
    
    res = minimize(f, [0,0], constraints=cons, args=(Sxx,Syy,Szz,Sxy,Syz,Szx,Sx,Sy,Sz), tol=1e-17)
    Squeez.append(res.fun*N/(Sx**2+Sy**2+Sz**2+1e-6))
    
    Arraytime.append(times[j])

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Spin Squeezing', fontsize = label_size)

plt.plot(Arraytime, -10*np.log(np.array(Squeez))/np.log(10),label="Spin Squeezing");

plt.legend()

## PIQS

In [ ]:
# TLS parameters
n_tls = N
system = Dicke(N = n_tls)
[jx, jy, jz] = jspin(N)
jp = jspin(N,"+")
jm = jp.dag()

system.hamiltonian = Omega * jx
system.collective_dephasing = 2*Gamma_phi
D_tls = system.liouvillian() 



# Light-matter coupling parameters

kappa_eff=2*kappa


nphot = int(2*N+2)  #modes of light
a = destroy(nphot)
h_int = (g_coef/np.sqrt(N)) *(tensor(a, jp)+tensor(a.dag(), jm))



# Photonic Liouvillian
c_ops_phot = [np.sqrt(kappa_eff) * a]

D_phot = liouvillian(Delta * a.dag()*a , c_ops_phot)

# Identity super-operators
nds = num_dicke_states(n_tls)
id_tls = to_super(qeye(nds))
id_phot = to_super(qeye(nphot))

# Define the total Liouvillian
D_int = -1j* spre(h_int) + 1j* spost(h_int)
D_tot = D_int + super_tensor(D_phot, id_tls) + super_tensor(id_phot, D_tls)

# Define operator in the total space
nphot_tot = tensor(a.dag()*a, qeye(nds))
adag_cuad_a_cuad = tensor(a.dag()*a.dag()*a*a, qeye(nds))

jx_tot=tensor(qeye(nphot), jx)
jy_tot=tensor(qeye(nphot), jy)
jz_tot=tensor(qeye(nphot), jz)

jxcuad_tot=tensor(qeye(nphot), jx*jx)
jycuad_tot=tensor(qeye(nphot), jy*jy)
jzcuad_tot=tensor(qeye(nphot), jz*jz)

jxysym=tensor(qeye(nphot), (jx*jy+jy*jx)/2)
jyzsym=tensor(qeye(nphot), (jy*jz+jz*jy)/2)
jzxsym=tensor(qeye(nphot), (jz*jx+jx*jz)/2)


In [ ]:
excited_state = excited(N)
ground_state = dicke(N, N/2, -N/2)

ground_phot = ket2dm(basis(nphot,0))  #vacuum
#rho0 = tensor(ground_phot, excited_state)
#rho0 = tensor(ground_phot, ground_state)



import scipy.special

rho_init=np.array(ground_state)


for n in range(N+1):
    for npr in range(N+1):
        sq_bin=np.sqrt(scipy.special.binom(N,n)*scipy.special.binom(N,npr))
        p_cos=np.power(np.cos(theta/2),n+npr)
        p_sin=np.power(np.sin(theta/2),2*N-n-npr)
        p_phi=np.exp(1j*phi*(npr-n))
        rho_init[n][npr]=sq_bin*p_cos*p_sin*p_phi

rho0=   tensor(ground_phot, Qobj(rho_init))     

In [ ]:
stepsPIQS=1000

t = np.linspace(0, T, stepsPIQS)
result1 = mesolve(D_tot, rho0, t, [], e_ops = [nphot_tot,jx_tot,jy_tot,jz_tot,adag_cuad_a_cuad,jxcuad_tot,jycuad_tot,jzcuad_tot,jxysym,jyzsym,jzxsym], 
                  options = Options(store_states=True))    #[nphot_tot,jx,jy,jz]
rhot_tot = result1.states
nphot_t = result1.expect[0]
jx_t = result1.expect[1]
jy_t = result1.expect[2]
jz_t = result1.expect[3]
adag_cuad_a_cuad_t=result1.expect[4]
jxx_t=result1.expect[5]
jyy_t=result1.expect[6]
jzz_t=result1.expect[7]

jxysym_t=result1.expect[8]
jyzsym_t=result1.expect[9]
jzxsym_t=result1.expect[10]

In [ ]:
matplotlib.rcParams['figure.figsize'] = (11.0, 8.2)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)

plt.plot(t, jx_t, 'k-', label='$S_X$ PIQS')
plt.plot(t, jy_t, 'r-', label='$S_y$ PIQS')
plt.plot(t, jz_t, 'g-', label='$S_z$ PIQS')

plt.title(r'Quantum Evolution of $\langle S_k \rangle(t)$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle S_k \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[0],"--",label="$S_x$ DDTWA")

plt.plot(times, Result_varDDTWA_EM[1],"--",label="$S_y$ DDTWA")

plt.plot(times, Result_varDDTWA_EM[2],"--",label="$S_z$ DDTWA")

plt.text(13, -2, "N="+str(N)+", $g$="+str(refg)+", $\Gamma_{\phi}$="+str(refgamma)+", $\Omega$="+str(refomega)+", $\kappa$="+str(refkappa)+", $\Delta$="+str(refdelta), style='italic',
        fontsize = 13 ,bbox={'facecolor': 'blue', 'alpha': 0.2, 'pad': 12})

plt.xlim([times[0],times[-1]])
#plt.plot(times, Result_varDDTWA_SH[0],"--",label="$S_x$ DDTWA SH")

#plt.plot(times, Result_varDDTWA_SH[1],"--",label="$S_y$ DDTWA SH")

#plt.plot(times, Result_varDDTWA_SH[2],"--",label="$S_z$ DDTWA SH")


#plt.legend(fontsize = label_size)
plt.legend(loc="upper right")
plt.show()
plt.close()

In [ ]:
j_max = N/2.
label_size = 20


fig3 = plt.figure(3)

plt.plot(t, jxx_t-(jx_t)**2, 'k-', label='$\Delta(S_x^2)$ PIQS')
plt.plot(t, jyy_t-(jy_t)**2, 'r-', label='$\Delta(S_y^2)$ PIQS')
plt.plot(t, jzz_t-(jz_t)**2, 'g-', label='$\Delta(S_z^2)$ PIQS')

plt.title(r'Quantum Evolution of $\Delta(S_k^2)(t)$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$ \Delta(S_k^2) (t)$', fontsize = label_size)



plt.plot(times,axEM,"-.",label="DDTWA $(\Delta S_x)^2 $ ")
plt.plot(times,ayEM,"-.",label="DDTWA $(\Delta S_y)^2 $ ")
plt.plot(times,azEM,"-.",label="DDTWA $(\Delta S_z)^2 $ ")

#plt.plot(times,axSH,"-.",label="DDTWA $(\Delta S_x)^2 $ SH")
#plt.plot(times,aySH,"-.",label="DDTWA $(\Delta S_y)^2 $ SH")
#plt.plot(times,azSH,"-.",label="DDTWA $(\Delta S_z)^2 $ SH")

plt.text(13, 1, "N="+str(N)+", $g$="+str(refg)+", $\Gamma_{\phi}$="+str(refgamma)+", $\Omega$="+str(refomega)+", $\kappa$="+str(refkappa)+", $\Delta$="+str(refdelta), style='italic',
        fontsize = 13 ,bbox={'facecolor': 'blue', 'alpha': 0.2, 'pad': 12})

plt.xlim([times[0],times[-1]])


#plt.legend(fontsize = label_size)
plt.legend(loc="upper right")
plt.show()
plt.close()

In [ ]:
j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
plt.plot(t, nphot_t, 'k-', label='$a^{\dagger}a$ PIQS')
plt.title(r'Quantum Dynamics Photon Number', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle a^{\dagger}a \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[9]-0.5,"--",label="$a^{\dagger}a$ DDTWA ")

#plt.plot(times, Result_varDDTWA_SH[9]-0.5,"--",label="$a^{\dagger}a$ DDTWA SH")


plt.text(12.3, 0.2, "N="+str(N)+", $g$="+str(refg)+", $\Gamma_{\phi}$="+str(refgamma)+", $\Omega$="+str(refomega)+", $\kappa$="+str(refkappa)+", $\Delta$="+str(refdelta), style='italic',
        fontsize = 13 ,bbox={'facecolor': 'blue', 'alpha': 0.2, 'pad': 12})

plt.xlim([times[0],times[-1]])



#plt.legend(fontsize = label_size)
plt.legend()
plt.show()
plt.close()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Quantum Dynamics $a^{\dagger}a^{\dagger}aa$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$\langle a^{\dagger}a^{\dagger} aa \rangle(t)$', fontsize = label_size)


plt.plot(times, Result_varDDTWA_EM[10],"--",label="$a^{\dagger}a^{\dagger}aa$ DDTWA ")
#plt.plot(times, Result_varDDTWA_SH[10],"--",label="$a^{\dagger}a^{\dagger}aa$ DDTWA SH")

plt.text(12.3, 0.2, "N="+str(N)+", $g$="+str(refg)+", $\Gamma_{\phi}$="+str(refgamma)+", $\Omega$="+str(refomega)+", $\kappa$="+str(refkappa)+", $\Delta$="+str(refdelta), style='italic',
        fontsize = 13 ,bbox={'facecolor': 'blue', 'alpha': 0.2, 'pad': 12})

plt.xlim([times[0],times[-1]])



plt.plot(t, adag_cuad_a_cuad_t, 'k-', label='$a^{\dagger}a^{\dagger}aa$ PIQS')

#plt.legend(fontsize = label_size)
plt.legend()
plt.show()
plt.close()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Quantum Dynamics of $g^{(2)}(0)$', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$g^{(2)}(0)$', fontsize = label_size)




g2_0=np.divide(adag_cuad_a_cuad_t,nphot_t**2);
g2_est_EM=np.divide(Result_varDDTWA_EM[10],(Result_varDDTWA_EM[9]-0.5)**2);
#g2_est_SH=np.divide(Result_varDDTWA_SH[10],(Result_varDDTWA_SH[9]-0.5)**2)



index=700
index1=30

plt.plot(t[index1:],g2_0[index1:], label='$g^{(2)}(0)$ PIQS')
plt.plot(times[index:],g2_est_EM[index:], label='$g^{(2)}(0)$ DDTWA ')
#plt.plot(times[index:],g2_est_SH[index:], label='$g^{(2)}(0)$ DDTWA SH')

plt.text(12.3, 1, "N="+str(N)+", $g$="+str(refg)+", $\Gamma_{\phi}$="+str(refgamma)+", $\Omega$="+str(refomega)+", $\kappa$="+str(refkappa)+", $\Delta$="+str(refdelta), style='italic',
        fontsize = 13 ,bbox={'facecolor': 'blue', 'alpha': 0.2, 'pad': 12})

plt.xlim([times[0],times[-1]])



#plt.legend(fontsize = label_size)
plt.legend(loc="upper right")
plt.show()
plt.close()


In [ ]:
f = lambda x, Sxx, Syy, Szz, Sxys, Syzs, Szxs, Sx, Sy, Sz: Sxx*power(cos(x[0])*sin(x[1]),2)+\
    Syy*power(sin(x[0])*sin(x[1]),2)+Szz*power(cos(x[1]),2)+Sxys*sin(2*x[0])*power(sin(x[1]),2)+\
    Syzs*sin(2*x[1])*sin(x[0])+Szxs*sin(2*x[1])*cos(x[0])


SquPIQS=[]

ArraytimePIQS=[]

separPIQS=1


for j in range(int(stepsPIQS/separPIQS)):
    
    j=separPIQS*j
        
    Sx= jx_t[j]
    Sy= jy_t[j]
    Sz= jz_t[j]
    
    Sxx= jxx_t[j]
    Syy= jyy_t[j]
    Szz= jzz_t[j]
    
    Sxy= jxysym_t[j]
    Syz= jyzsym_t[j]
    Szx= jzxsym_t[j]
    
    cons = ({'type': 'eq', 'fun': lambda x: Sx*cos(x[0])*sin(x[1])+Sy*sin(x[0])*sin(x[1])+Sz*cos(x[1])})
    
    res = minimize(f, [0,0], constraints=cons, args=(Sxx,Syy,Szz,Sxy,Syz,Szx,Sx,Sy,Sz), tol=1e-17)
    SquPIQS.append(res.fun*N/(Sx**2+Sy**2+Sz**2+1e-6))
    
    ArraytimePIQS.append(t[j])

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 6.5)

j_max = N/2.
label_size = 20


fig3 = plt.figure(3)
#plt.plot(t, nphot_t, 'k-', label='Time evolution')
plt.title(r'Spin Squeezing Dynamics', fontsize = label_size)
plt.xlabel(r'$t$', fontsize = label_size)
#plt.ylabel(r'$\langle a^\dagger a\rangle(t)$', fontsize = label_size)
plt.ylabel(r'$-10\log_{10}(\xi^2)$', fontsize = label_size)




index=700
index1=30

plt.plot(ArraytimePIQS,-10*np.log(np.array(SquPIQS))/np.log(10),label="Numerical")
plt.plot(Arraytime,-10*np.log(np.array(Squeez))/np.log(10),label="DDTWA")
#plt.plot(times[index:],g2_est_SH[index:], label='$g^{(2)}(0)$ DDTWA SH')

plt.text(11.6, -1, "N="+str(N)+", $g$="+str(refg)+", $\Gamma_{\phi}$="+str(refgamma)+", $\Omega$="+str(refomega)+", $\kappa$="+str(refkappa)+", $\Delta$="+str(refdelta), style='italic',
        fontsize = 13 ,bbox={'facecolor': 'blue', 'alpha': 0.2, 'pad': 12})

plt.xlim([times[0],times[-1]])



#plt.legend(fontsize = label_size)
plt.legend(loc="lower right")
plt.show()
plt.close()
